<a href="https://colab.research.google.com/github/bikash119/learn_tensorflow/blob/main/nlp_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character RNN

## Creating the Training Dataset

In [1]:
import tensorflow as tf

filepath= tf.keras.utils.get_file("shakespeare_txt",
                                  origin="https://homl.info/shakespeare")
with open(filepath) as fp:
  shakespeare_tex= fp.read()

1115394/1115394 [==============================] - 0s 0us/step


In [2]:
print(shakespeare_tex[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


## Perform Text Vectorization

In [3]:
# Perform character text vectorization
text_vec_layer= tf.keras.layers.TextVectorization(split="character",
                                                  standardize="lower")
text_vec_layer.adapt([shakespeare_tex])

In [4]:
sample_sentence=" I am here to learn nlp"
print(len(sample_sentence))
## Vectorize the sample sentence
text_vec_layer([sample_sentence])

23


<tf.Tensor: shape=(1, 23), dtype=int64, numpy=
array([[ 2,  7,  2,  6, 16,  2,  8,  3, 10,  3,  2,  4,  5,  2, 13,  3,
         6, 10, 11,  2, 11, 13, 24]])>

In [5]:
tokens= text_vec_layer([shakespeare_tex])[0]
# Print some text from original text
print(f" Original Text : {shakespeare_tex[:80]} ")
# Print the vectorized text
print(f" Vectorized tokens: {tokens[:80]}")
print(len(tokens))

 Original Text : First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak. 
 Vectorized tokens: [21  7 10  9  4  2 20  7  4  7 37  3 11 25 12 23  3 21  5 10  3  2 18  3
  2 24 10  5 20  3  3 14  2  6 11 17  2 21 15 10  4  8  3 10 19  2  8  3
  6 10  2 16  3  2  9 24  3  6 26 28 12 12  6 13 13 25 12  9 24  3  6 26
 19  2  9 24  3  6 26 28]
1115394


In [6]:
## Unique token characters in vocabulary
chars_in_vocab= text_vec_layer.get_vocabulary()
top_5_chars= chars_in_vocab[:5]
bottom_5_chars= chars_in_vocab[-5:]
print(f" Number of characters in vocab :{text_vec_layer.vocabulary_size()}")
print(f" top 5 common tokens: {top_5_chars}")
print(f" bottom 5 common tokens: {bottom_5_chars}")

 Number of characters in vocab :41
 top 5 common tokens: ['', '[UNK]', ' ', 'e', 't']
 bottom 5 common tokens: ['x', 'z', '3', '&', '$']


In [7]:
#tokens -= 2 # Remove the 0 (pad) and 1(UNK) token
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size= len(tokens)
print(f" dataset size : {len(tokens)}")

 dataset size : 1115394


## Perpare Dataset

In [8]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
  ds= tf.data.Dataset.from_tensor_slices(sequence)
  ds= ds.window(length+1, shift=1, drop_remainder=True)
  ds= ds.flat_map(lambda window_ds: window_ds.batch(length+1))
  if shuffle:
    ds= ds.shuffle(buffer_size=100_000,seed=seed)
  ds= ds.batch(batch_size)
  return ds.map(lambda window: (window[:, :-1], window[:,1:])).prefetch(1)


In [9]:
length= 100
tf.random.set_seed(42)
train_set= to_dataset(tokens[:100_000],length=length
                      , shuffle=True,seed=42)
valid_set= to_dataset(tokens[1_000_000:1_060_000],length=length)
test_set= to_dataset(tokens[1_600_000:],length=length)

## Creating an Embedding using an Embedding Layer

In [10]:
tf.random.set_seed(42)
embedding_layer = tf.keras.layers.Embedding(input_dim=n_tokens
                                            ,output_dim=16
                                            ,input_length=length)
embedding_layer

In [11]:
sample_sentenc= " I am here to learn NLP"
print(f" Sample Sentenc : {sample_sentence}")
tokens_of_sample_sentence= text_vec_layer(sample_sentence)
print(f" length of tokens from TextVectorizer for sample sentence: {len(tokens_of_sample_sentence)}")
embeddings_of_sample_sentence= embedding_layer(tokens_of_sample_sentence)
print(f" Shape of Embeddings from embeding layer for sample sentence: {embeddings_of_sample_sentence.shape}")


 Sample Sentenc :  I am here to learn nlp
 length of tokens from TextVectorizer for sample sentence: 23
 Shape of Embeddings from embeding layer for sample sentence: (23, 16)


## Building and Training the Char-RNN model


In [ ]:
char_model= tf.keras.Sequential([
    embedding_layer
    ,tf.keras.layers.GRU(128,return_sequences=True)
    ,tf.keras.layers.Dense(n_tokens,activation="softmax")
])

char_model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy
                   ,optimizer=tf.keras.optimizers.Nadam()
                   ,metrics=['accuracy'])
model_ckpt= tf.keras.callbacks.ModelCheckpoint("my_shakespeare_model"
                                               ,monitor="val_accuracy"
                                               ,save_best_only=True)
history= char_model.fit(train_set
                        ,validation_data=valid_set
                        ,epochs=1
                        ,callbacks=[model_ckpt])

      1/Unknown - 25s 25s/step - loss: 3.6633 - accuracy: 0.0131

InvalidArgumentError: ignored

# Tweet Categorization
Text-based Tweets about natural disasters.

## Download and unzip the data

In [1]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
import zipfile
import os
def unzip_data(file, remove_zip_file=True):
  """
  unizp the zipped file and optionally removes the original zipefile
  """
  zipref= zipfile.ZipFile(file)
  zipref.extractall()
  os.remove(file)


--2023-08-19 06:19:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.96.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.27MB/s    in 0.5s    

2023-08-19 06:19:57 (1.27 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [2]:
unzip_data("/content/nlp_getting_started.zip")

## Load the data as pandas dataframe

In [3]:
import pandas as pd

train_df= pd.read_csv("/content/train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Split the dataset into train and test

In [4]:
from sklearn.model_selection import train_test_split
# the ouput tuple is of the format (X_train, X_valid, y_train, y_valid)
train_sent, val_sent,train_target,val_target= train_test_split(train_df["text"].to_numpy()
                                                           ,train_df["target"].to_numpy()
                                                           ,test_size=0.1
                                                           ,random_state=42)
print(f"""
training feature rows : {len(train_sent)}
training target rows : {len(train_target)}
validation feature rows: {len(val_target)}
validation target rows: {len(val_target)}
      """)


training feature rows : 6851
training target rows : 6851
validation feature rows: 762
validation target rows: 762
      


## Text Vectorization
Convert the text to numbers

In [5]:
import tensorflow as tf
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer= tf.keras.layers.TextVectorization(max_tokens=max_vocab_length
                                                  ,output_sequence_length=max_length
                                                  )
text_vectorizer.adapt(train_sent)

import random
random_sentence= random.choice(train_sent)
print(f" Random Sentence from training set :\n {random_sentence}\n")
vectorized_random_sentence= text_vectorizer(random_sentence)
print(f" Vectorized form of random sentence : \n { vectorized_random_sentence}")

 Random Sentence from training set :
 Shed innocent blood of their sons and daughters and the land was polluted Psalms 106:38  Help stop the sin of abortion.

 Vectorized form of random sentence : 
 [4624 5325  348    6  114 4558    7 3064    7    2  549   23 9695 9502
    1]


In [6]:
print(f" Size of the vocabulary : {text_vectorizer.vocabulary_size()}")

 Size of the vocabulary : 10000


## Model Metrics function
Function to evaluate a model accuracy, f1 ,precision, recall


In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

## Modelling a text dataset

### Naive Bayes ( Baseline )

In [8]:
# Build the model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0= Pipeline([
    ("tfidf",TfidfVectorizer())
    ,("clf",MultinomialNB())
])

# Fit the model
model_0.fit(train_sent,train_target)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [9]:
# Score the model
model_0_results= model_0.score(val_sent,val_target)
print(f" Accuracy from baseline model : {model_0_results*100:.3f}%")

 Accuracy from baseline model : 77.822%


In [10]:
# Make predictions
model_0_preds= model_0.predict(val_sent)

print(f" Target Labels : {val_target[:10]}")
import numpy as np

np.isclose(val_target[:10],model_0_preds[:10])

 Target Labels : [1 0 1 0 0 0 1 1 0 1]


array([False,  True, False,  True,  True,  True, False, False,  True,
        True])

In [11]:
calculate_results(val_target,model_0_preds)

{'accuracy': 77.82152230971128,
 'precision': 0.792992256322435,
 'recall': 0.7782152230971129,
 'f1': 0.7703527809038113}

### Dense Model (Model 1)


####  Sequential API

##### Embedding Layer
This will create learnable embeddings for our text

In [12]:
# input_dim= vocabulary size
# output_dim= size of the vectors for each element in vocabulary
# input_length= Length of input sequences, when it is constant
embedding_layer= tf.keras.layers.Embedding(input_dim=max_vocab_length
                                           ,output_dim=128
                                           ,input_length=max_length)

import random
random_sentence= random.choice(train_sent)
print(f" Random Sentence from training set :\n {random_sentence}")

vectorized_random_sentence= text_vectorizer(random_sentence)
print(f" \nVectorized form of random sentence : \n { vectorized_random_sentence}")

embeddings_of_sample_sentence= embedding_layer(vectorized_random_sentence)
print(f" \nEmbeddings random sentence : \n { embeddings_of_sample_sentence}")
print(f" \n Shape of embeddings : {embeddings_of_sample_sentence.shape}")

 Random Sentence from training set :
 The thunder shook my house woke my sister and made car alarms go off ????
 
Vectorized form of random sentence : 
 [   2  281 3420   13  290 1580   13 1766    7  286  120 4151  111   93
    0]
 
Embeddings random sentence : 
 [[-0.01007521 -0.02513694  0.04380209 ...  0.03106899  0.0045206
  -0.0080335 ]
 [ 0.04133676 -0.02438538 -0.01256455 ... -0.004109   -0.00314444
   0.02058831]
 [-0.01298745 -0.02286965 -0.0107521  ...  0.04048712  0.03673976
  -0.02227726]
 ...
 [ 0.01340124 -0.03621493 -0.046087   ...  0.0178371  -0.01586359
  -0.04420078]
 [ 0.04104669  0.00935958  0.0095611  ...  0.01601711  0.02357067
   0.00550453]
 [-0.04736855  0.04581997 -0.02523478 ...  0.04827455  0.04164732
  -0.00416782]]
 
 Shape of embeddings : (15, 128)


##### Build , Train & Evaluate Model

In [13]:
tf.random.set_seed(42)
# Build the model using Sequential API
import tensorflow as tf
from tensorflow.keras import layers
model_1_seq= tf.keras.Sequential([
    layers.Input(shape=(1,),dtype="string")
    ,text_vectorizer
    ,embedding_layer
    ,layers.GlobalAveragePooling1D()
    ,layers.Dense(1,activation="sigmoid")
])
model_1_seq.compile(loss=tf.keras.losses.binary_crossentropy
                    ,optimizer=tf.keras.optimizers.Adam()
                    ,metrics=["accuracy"])
model_1_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Train the model
model_1_seq_hist= model_1_seq.fit(train_sent,train_target
                                  ,validation_data=(val_sent,val_target)
                                  ,epochs=3)

Epoch 1/3
215/215 [==============================] - 11s 39ms/step - loss: 0.6114 - accuracy: 0.6887 - val_loss: 0.5413 - val_accuracy: 0.7625
Epoch 2/3
215/215 [==============================] - 8s 35ms/step - loss: 0.4395 - accuracy: 0.8170 - val_loss: 0.4831 - val_accuracy: 0.7887
Epoch 3/3
215/215 [==============================] - 7s 31ms/step - loss: 0.3451 - accuracy: 0.8622 - val_loss: 0.4749 - val_accuracy: 0.7874


In [15]:
# Evaluate the model
model_1_seq.evaluate(val_sent,val_target)

24/24 [==============================] - 0s 7ms/step - loss: 0.4749 - accuracy: 0.7874


[0.47487494349479675, 0.787401556968689]

In [16]:
# Do Prediction
model_1_seq_preds_proba= model_1_seq.predict(val_sent)
model_1_seq_preds= tf.squeeze(tf.round(model_1_seq_preds_proba))

24/24 [==============================] - 1s 3ms/step


In [17]:
calculate_results(y_true=val_target
                  ,y_pred=model_1_seq_preds)

{'accuracy': 78.74015748031496,
 'precision': 0.7897763444463061,
 'recall': 0.7874015748031497,
 'f1': 0.7844490087488502}

#### Functional API

##### Embedding Layer
This will create learnable embeddings for our text

In [18]:
# input_dim= vocabulary size
# output_dim= size of the vectors for each element in vocabulary
# input_length= Length of input sequences, when it is constant
embedding_layer= tf.keras.layers.Embedding(input_dim=max_vocab_length
                                           ,output_dim=128
                                           ,input_length=max_length)

import random
random_sentence= random.choice(train_sent)
print(f" Random Sentence from training set :\n {random_sentence}")

vectorized_random_sentence= text_vectorizer(random_sentence)
print(f" \nVectorized form of random sentence : \n { vectorized_random_sentence}")

embeddings_of_sample_sentence= embedding_layer(vectorized_random_sentence)
print(f" \nEmbeddings random sentence : \n { embeddings_of_sample_sentence}")
print(f" \n Shape of embeddings : {embeddings_of_sample_sentence.shape}")

 Random Sentence from training set :
 The fact that the atomic bombs were called 'Little Boy' and 'Fat man' says a lot about the mentality that went into the attack.
 
Vectorized form of random sentence : 
 [   2 1299   16    2  214 1027   68  778  245  332    7 1533   96  273
    3]
 
Embeddings random sentence : 
 [[-0.0386566   0.0039768  -0.04148995 ...  0.02769985 -0.02348822
  -0.02563067]
 [-0.01421382  0.03974347 -0.03256079 ... -0.03627352  0.02616512
  -0.00466117]
 [ 0.01765224 -0.01619581  0.04560056 ... -0.02538377  0.0025444
  -0.00897261]
 ...
 [-0.005717   -0.04298306  0.0079469  ...  0.00591335 -0.02090634
  -0.04462837]
 [-0.03524525 -0.01381073  0.0404788  ...  0.00683331 -0.04372438
  -0.04445002]
 [ 0.03734577 -0.04282352  0.02210995 ... -0.04578521  0.0003613
   0.01329196]]
 
 Shape of embeddings : (15, 128)


##### Build , Train, Evaluate Model

In [19]:
tf.random.set_seed(42)
import tensorflow as tf
from tensorflow.keras import layers

inputs= layers.Input(shape=(1,), dtype="string")
x= text_vectorizer(inputs)
x= embedding_layer(x)
x= layers.GlobalAveragePooling1D(name="globalavgpooling")(x)
outputs= layers.Dense(1,activation="sigmoid",name="dense")(x)
model_1_func= tf.keras.Model(inputs=inputs
                             ,outputs= outputs
                             ,name="dense_model_1_func")

model_1_func.compile(loss=tf.keras.losses.binary_crossentropy
                     ,optimizer=tf.keras.optimizers.Adam()
                     ,metrics=["accuracy"])

model_1_func.summary()

Model: "dense_model_1_func"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 globalavgpooling (GlobalAve  (None, 128)              0         
 ragePooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,

In [20]:
model_1_func_hist= model_1_func.fit(train_sent,train_target
                                    ,validation_data=(val_sent,val_target)
                                    ,epochs=3)

Epoch 1/3
215/215 [==============================] - 5s 18ms/step - loss: 0.6093 - accuracy: 0.6965 - val_loss: 0.5402 - val_accuracy: 0.7651
Epoch 2/3
215/215 [==============================] - 4s 18ms/step - loss: 0.4388 - accuracy: 0.8178 - val_loss: 0.4829 - val_accuracy: 0.7913
Epoch 3/3
215/215 [==============================] - 4s 20ms/step - loss: 0.3446 - accuracy: 0.8625 - val_loss: 0.4748 - val_accuracy: 0.7887


In [21]:
# Evaluate the model
model_1_func.evaluate(val_sent,val_target)

24/24 [==============================] - 0s 2ms/step - loss: 0.4748 - accuracy: 0.7887


[0.47482219338417053, 0.7887139320373535]

In [22]:
model_1_func_proba= model_1_func.predict(val_sent)
model_1_func_preds= tf.squeeze(tf.round(model_1_func_proba))


24/24 [==============================] - 0s 2ms/step


In [23]:
calculate_results(y_true=val_target,y_pred=model_1_func_preds)

{'accuracy': 78.87139107611549,
 'precision': 0.7913065706940227,
 'recall': 0.7887139107611548,
 'f1': 0.785702921924679}

### Recurrent Neural Network (LSTM)

#### Sequential API

##### Embedding Layer

In [24]:
import tensorflow as tf
from tensorflow.keras import layers
# Since the embeddings are learnable we will have to create the embedding for every
# model
embedding_layer_rnn= layers.Embedding(input_dim=max_vocab_length
                                      ,output_dim=128
                                      ,input_length=max_length)

##### Build, Train, Evaluate the model

In [25]:
# Build the model
tf.random.set_seed(42)
model_2_seq= tf.keras.Sequential([
    layers.Input(shape=(1,),dtype="string")
    ,text_vectorizer
    ,embedding_layer_rnn
    ,layers.LSTM(64)
    ,layers.Dense(1,activation="sigmoid")
])
model_2_seq.compile(loss=tf.keras.losses.binary_crossentropy
                    ,optimizer=tf.keras.optimizers.Adam()
                    ,metrics=["accuracy"])
model_2_seq.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Train the model
model_2_seq_hist= model_2_seq.fit(train_sent,train_target
                                  ,validation_data=(val_sent,val_target)
                                  ,epochs=3)

Epoch 1/3
215/215 [==============================] - 11s 34ms/step - loss: 0.5139 - accuracy: 0.7450 - val_loss: 0.4660 - val_accuracy: 0.7913
Epoch 2/3
215/215 [==============================] - 8s 35ms/step - loss: 0.3155 - accuracy: 0.8720 - val_loss: 0.5228 - val_accuracy: 0.7808
Epoch 3/3
215/215 [==============================] - 7s 30ms/step - loss: 0.2183 - accuracy: 0.9206 - val_loss: 0.5257 - val_accuracy: 0.7703


In [27]:
# Evaluate the model
model_2_seq.evaluate(val_sent,val_target)

24/24 [==============================] - 0s 5ms/step - loss: 0.5257 - accuracy: 0.7703


[0.5256800651550293, 0.7703412175178528]

In [28]:
# Do prediction
model_2_pred_proba= model_2_seq.predict(val_sent)
model_2_preds= tf.squeeze(tf.round(model_2_pred_proba))

24/24 [==============================] - 1s 5ms/step


In [29]:
# Calculate results
calculate_results(y_true=val_target,y_pred=model_2_preds)

{'accuracy': 77.03412073490814,
 'precision': 0.7707448774088915,
 'recall': 0.7703412073490814,
 'f1': 0.7680081280884467}

#### Functional API

##### Embedding Layer

In [30]:
import tensorflow as tf
from tensorflow.keras import layers

embedding_layer_rnn2= layers.Embedding(input_dim=max_vocab_length
                                       ,output_dim=128
                                       ,input_length=25)

##### Build, Train & Evaluate

In [31]:
## Build the model
tf.random.set_seed(42)
inputs= layers.Input(shape=(1,),dtype="string")
x= text_vectorizer(inputs)
x= embedding_layer_rnn2(x)
x= layers.LSTM(64)(x)
outputs= layers.Dense(1,activation="sigmoid")(x)
model_2_func= tf.keras.Model(inputs= inputs
                             ,outputs=outputs
                             ,name="model_2_rnn")

model_2_func.compile(loss=tf.keras.losses.binary_crossentropy
                     ,optimizer=tf.keras.optimizers.Adam()
                     ,metrics=["accuracy"])

model_2_func.summary()

Model: "model_2_rnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_____________________________________________

In [32]:
# Train the model
model_2_func_hist= model_2_func.fit(train_sent,train_target
                                    ,validation_data=(val_sent,val_target)
                                    ,epochs=3)

Epoch 1/3
215/215 [==============================] - 11s 41ms/step - loss: 0.5120 - accuracy: 0.7475 - val_loss: 0.4625 - val_accuracy: 0.7913
Epoch 2/3
215/215 [==============================] - 7s 34ms/step - loss: 0.3144 - accuracy: 0.8727 - val_loss: 0.5273 - val_accuracy: 0.7874
Epoch 3/3
215/215 [==============================] - 7s 34ms/step - loss: 0.2155 - accuracy: 0.9209 - val_loss: 0.5357 - val_accuracy: 0.7808


In [33]:
# Evaluate the model
model_2_func.evaluate(val_sent,val_target)

24/24 [==============================] - 0s 8ms/step - loss: 0.5357 - accuracy: 0.7808


[0.5356898903846741, 0.7808399200439453]

In [34]:
# Predict
model_2_func_preds_proba= model_2_func.predict(val_sent)
model_2_func_preds= tf.squeeze(tf.round(model_2_func_preds_proba))

24/24 [==============================] - 1s 6ms/step


In [35]:
# Calculate results
calculate_results(y_true=val_target,y_pred=model_2_func_preds)

{'accuracy': 78.08398950131233,
 'precision': 0.7828035148534721,
 'recall': 0.7808398950131233,
 'f1': 0.7778748302844208}

### Recurrent Neural Network (GRU)

#### Sequential API

##### Embedding Layer

In [36]:
import tensorflow as tf
from tensorflow.keras import layers
# Since the embeddings are learnable we will have to create the embedding for every
# model
embedding_layer_gru= layers.Embedding(input_dim=max_vocab_length
                                      ,output_dim=128
                                      ,input_length=max_length)

##### Build, Train, Evaluate the model

In [37]:
# Build the model
tf.random.set_seed(42)
model_3_seq= tf.keras.Sequential([
    layers.Input(shape=(1,),dtype="string")
    ,text_vectorizer
    ,embedding_layer_gru
    ,layers.GRU(64)
    ,layers.Dense(1,activation="sigmoid")
])
model_3_seq.compile(loss=tf.keras.losses.binary_crossentropy
                    ,optimizer=tf.keras.optimizers.Adam()
                    ,metrics=["accuracy"])
model_3_seq.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Train the model
model_3_seq_hist= model_3_seq.fit(train_sent,train_target
                                  ,validation_data=(val_sent,val_target)
                                  ,epochs=3)

Epoch 1/3
215/215 [==============================] - 11s 36ms/step - loss: 0.5291 - accuracy: 0.7297 - val_loss: 0.4674 - val_accuracy: 0.7979
Epoch 2/3
215/215 [==============================] - 8s 38ms/step - loss: 0.3168 - accuracy: 0.8713 - val_loss: 0.5162 - val_accuracy: 0.7848
Epoch 3/3
215/215 [==============================] - 8s 36ms/step - loss: 0.2184 - accuracy: 0.9165 - val_loss: 0.5215 - val_accuracy: 0.7795


In [39]:
# Evaluate the model
model_3_seq.evaluate(val_sent,val_target)

24/24 [==============================] - 0s 4ms/step - loss: 0.5215 - accuracy: 0.7795


[0.5214547514915466, 0.7795275449752808]

In [40]:
# Do prediction
model_3_pred_proba= model_3_seq.predict(val_sent)
model_3_preds= tf.squeeze(tf.round(model_3_pred_proba))

24/24 [==============================] - 1s 4ms/step


In [41]:
# Calculate results
calculate_results(y_true=val_target,y_pred=model_3_preds)

{'accuracy': 77.95275590551181,
 'precision': 0.7812950247858184,
 'recall': 0.7795275590551181,
 'f1': 0.7766229620954818}

#### Functional API

##### Embedding Layer

In [42]:
import tensorflow as tf
from tensorflow.keras import layers

embedding_layer_gru2= layers.Embedding(input_dim=max_vocab_length
                                       ,output_dim=128
                                       ,input_length=25)

##### Build, Train & Evaluate

In [43]:
## Build the model
tf.random.set_seed(42)
inputs= layers.Input(shape=(1,),dtype="string")
x= text_vectorizer(inputs)
x= embedding_layer_gru2(x)
x= layers.LSTM(64)(x)
outputs= layers.Dense(1,activation="sigmoid")(x)
model_3_func= tf.keras.Model(inputs= inputs
                             ,outputs=outputs
                             ,name="model_3_gru")

model_3_func.compile(loss=tf.keras.losses.binary_crossentropy
                     ,optimizer=tf.keras.optimizers.Adam()
                     ,metrics=["accuracy"])

model_3_func.summary()

Model: "model_3_gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_____________________________________________

In [44]:
# Train the model
model_3_func_hist= model_3_func.fit(train_sent,train_target
                                    ,validation_data=(val_sent,val_target)
                                    ,epochs=3)

Epoch 1/3
215/215 [==============================] - 11s 36ms/step - loss: 0.5140 - accuracy: 0.7459 - val_loss: 0.4627 - val_accuracy: 0.8018
Epoch 2/3
215/215 [==============================] - 8s 35ms/step - loss: 0.3152 - accuracy: 0.8733 - val_loss: 0.5231 - val_accuracy: 0.7913
Epoch 3/3
215/215 [==============================] - 7s 31ms/step - loss: 0.2180 - accuracy: 0.9196 - val_loss: 0.5172 - val_accuracy: 0.7703


In [45]:
# Evaluate the model
model_3_func.evaluate(val_sent,val_target)

24/24 [==============================] - 0s 5ms/step - loss: 0.5172 - accuracy: 0.7703


[0.5172091722488403, 0.7703412175178528]

In [46]:
# Predict
model_3_func_preds_proba= model_3_func.predict(val_sent)
model_3_func_preds= tf.squeeze(tf.round(model_3_func_preds_proba))

24/24 [==============================] - 1s 5ms/step


In [47]:
# Calculate results
calculate_results(y_true=val_target,y_pred=model_3_func_preds)

{'accuracy': 77.03412073490814,
 'precision': 0.7721862413486815,
 'recall': 0.7703412073490814,
 'f1': 0.7670683933963902}

### Model 4: Bidirectional RNN

#### Sequential API

##### Emdedding Layer

In [48]:
import tensorflow as tf
from tensorflow.keras import layers
embedding_layer_birnn= layers.Embedding(input_dim=max_vocab_length
                                        ,output_dim=128
                                        ,input_length=max_length)

##### Build, Train, Evaluate Model

In [49]:
# Build the model
model_4_seq= tf.keras.Sequential([
    layers.Input(shape=(1,),dtype="string")
    ,text_vectorizer
    ,embedding_layer_birnn
    ,layers.Bidirectional(layers.LSTM(64))
    ,layers.Dense(1,activation="sigmoid")
])
model_4_seq.compile(loss=tf.keras.losses.binary_crossentropy
                    ,optimizer=tf.keras.optimizers.Adam()
                    ,metrics=["accuracy"])

model_4_seq.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_6 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Train the model
model_4_seq_hist= model_4_seq.fit(train_sent,train_target
                                  ,validation_data=(val_sent,val_target)
                                  ,epochs=3)

Epoch 1/3
215/215 [==============================] - 16s 51ms/step - loss: 0.5104 - accuracy: 0.7462 - val_loss: 0.4612 - val_accuracy: 0.8005
Epoch 2/3
215/215 [==============================] - 9s 42ms/step - loss: 0.3134 - accuracy: 0.8716 - val_loss: 0.4928 - val_accuracy: 0.7874
Epoch 3/3
215/215 [==============================] - 10s 47ms/step - loss: 0.2104 - accuracy: 0.9222 - val_loss: 0.5507 - val_accuracy: 0.7861


In [51]:
# evaluate the model
model_4_seq.evaluate(val_sent,val_target)

24/24 [==============================] - 0s 7ms/step - loss: 0.5507 - accuracy: 0.7861


[0.5507320761680603, 0.7860892415046692]

In [52]:
# Predict
model_4_seq_pred_proba= model_4_seq.predict(val_sent)
model_4_seq_pred= tf.squeeze(tf.round(model_4_seq_pred_proba))

24/24 [==============================] - 1s 7ms/step


In [53]:
# Calculate result
calculate_results(y_true=val_target,y_pred=model_4_seq_pred)

{'accuracy': 78.60892388451444,
 'precision': 0.7909099906966713,
 'recall': 0.7860892388451444,
 'f1': 0.7820422109103825}

#### Functional API

##### Embedding Layer

In [54]:
import tensorflow as tf
from tensorflow.keras import layers
embedding_layer_birnn2= layers.Embedding(input_dim=max_vocab_length
                                        ,output_dim= 128
                                        ,input_length=max_length)

##### Train, Build & Evaluate Model

In [55]:
# Build the model
inputs= layers.Input(shape=(1,),dtype="string")
x= text_vectorizer(inputs)
x= embedding_layer_birnn2(x)
x= layers.Bidirectional(layers.LSTM(64))(x)
outputs= layers.Dense(1,activation="sigmoid")(x)
model_4_func= tf.keras.Model(inputs=inputs
                             ,outputs=outputs
                             ,name="model_4_bidirectional_birnn2")

model_4_func.compile(loss=tf.keras.losses.binary_crossentropy
                     ,optimizer=tf.keras.optimizers.Adam()
                     ,metrics=["accuracy"])

model_4_func.summary()

Model: "model_4_bidirectional_birnn2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_7 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable para

In [56]:
# Train the model
model_4_func_hist= model_4_func.fit(train_sent,train_target
                                    ,validation_data=(val_sent,val_target)
                                    ,epochs=3)

Epoch 1/3
215/215 [==============================] - 16s 50ms/step - loss: 0.5123 - accuracy: 0.7432 - val_loss: 0.4619 - val_accuracy: 0.7992
Epoch 2/3
215/215 [==============================] - 8s 38ms/step - loss: 0.3123 - accuracy: 0.8716 - val_loss: 0.4937 - val_accuracy: 0.7953
Epoch 3/3
215/215 [==============================] - 9s 44ms/step - loss: 0.2060 - accuracy: 0.9228 - val_loss: 0.5660 - val_accuracy: 0.7795


In [57]:
# Evaluate
model_4_func.evaluate(val_sent,val_target)

24/24 [==============================] - 0s 7ms/step - loss: 0.5660 - accuracy: 0.7795


[0.5659992098808289, 0.7795275449752808]

In [58]:
# Predict
model_4_func_pred_proba= model_4_func.predict(val_sent)
model_4_func_pred= tf.squeeze(tf.round(model_4_func_pred_proba))

24/24 [==============================] - 1s 8ms/step


In [59]:
# Calculate results
calculate_results(y_true=val_target,y_pred=model_4_func_pred)

{'accuracy': 77.95275590551181,
 'precision': 0.7851161266397013,
 'recall': 0.7795275590551181,
 'f1': 0.7748890479599141}